### Environment setup

In [1]:
from pynwb import NWBHDF5IO
from scipy import interpolate
import numpy as np
import matplotlib.pyplot as plt

### Read NWB files

In [2]:
lfp_filepath = "../../../data/GLO/probeA_lfp.nwb"
stim_filepath = "../../../data/GLO/spike_times.nwb"

In [3]:
lfp_io = NWBHDF5IO(lfp_filepath, mode="r", load_namespaces=True)
lfp_file = lfp_io.read()
stim_io = NWBHDF5IO(stim_filepath, mode="r", load_namespaces=True)
stim_file = stim_io.read() 

C:\Users\carter.peene\AppData\Local\Programs\Python\Python39\lib\site-packages\hdmf\spec\namespace.py:531: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.5.0 because version 1.5.1 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
C:\Users\carter.peene\AppData\Local\Programs\Python\Python39\lib\site-packages\hdmf\spec\namespace.py:531: UserWarning: Ignoring cached namespace 'core' version 2.4.0 because version 2.5.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
C:\Users\carter.peene\AppData\Local\Programs\Python\Python39\lib\site-packages\hdmf\spec\namespace.py:531: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.1.0 because version 0.2.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


### Extract stimulus times

In [4]:
stimulus_names = list(stim_file.intervals.keys())
print(stimulus_names)

['create_receptive_field_mapping_presentations', 'init_grating_presentations', 'init_intermission_presentations', 'invalid_times', 'spontaneous_presentations']


In [5]:
stim_name = "init_grating_presentations"
stim_table = stim_file.intervals[stim_name]

In [6]:
stim_table[0:50]

,start_time,stop_time,stimulus_name,stimulus_block,contrast,temporal_frequency,spatial_frequency,orientation,color,mask,opacity,size,units,stimulus_index,phase,tags,timeseries
id,,,,,,,,,,,,,,,,,
0,178.848010,179.348411,init_grating,1.0,0.8,4.0,0.04,135.0,"[1.0, 1.0, 1.0]",None,1.0,"[1000.0, 1000.0]",deg,1.0,0.0,[stimulus_time_interval],"[(2, 1, timestamps pynwb.base.TimeSeries at 0x..."
1,179.848811,180.349251,init_grating,1.0,0.8,4.0,0.04,135.0,"[1.0, 1.0, 1.0]",None,1.0,"[1000.0, 1000.0]",deg,1.0,0.0,[stimulus_time_interval],"[(3, 1, timestamps pynwb.base.TimeSeries at 0x..."
2,180.849690,181.350086,init_grating,1.0,0.8,4.0,0.04,135.0,"[1.0, 1.0, 1.0]",None,1.0,"[1000.0, 1000.0]",deg,1.0,0.0,[stimulus_time_interval],"[(4, 1, timestamps pynwb.base.TimeSeries at 0x..."
3,181.850481,182.350931,init_grating,1.0,0.8,4.0,0.04,45.0,"[1.0, 1.0, 1.0]",None,1.0,"[1000.0, 1000.0]",deg,1.0,0.0,[stimulus_time_interval],"[(5, 1, timestamps pynwb.base.TimeSeries at 0x..."
4,182.851380,183.351771,init_grating,1.0,NaN,NaN,NaN,NaN,"[1.0, 1.0, 1.0]",None,1.0,"[1000.0, 1000.0]",deg,1.0,N/A,[stimulus_time_interval],"[(6, 1, timestamps pynwb.base.TimeSeries at 0x..."
5,183.852161,184.352606,init_grating,1.0,0.8,4.0,0.04,135.0,"[1.0, 1.0, 1.0]",None,1.0,"[1000.0, 1000.0]",deg,1.0,0.0,[stimulus_time_interval],"[(7, 1, timestamps pynwb.base.TimeSeries at 0x..."
6,184.853050,185.353451,init_grating,1.0,0.8,4.0,0.04,135.0,"[1.0, 1.0, 1.0]",None,1.0,"[1000.0, 1000.0]",deg,1.0,0.0,[stimulus_time_interval],"[(8, 1, timestamps pynwb.base.TimeSeries at 0x..."
7,185.853851,186.354301,init_grating,1.0,0.8,4.0,0.04,135.0,"[1.0, 1.0, 1.0]",None,1.0,"[1000.0, 1000.0]",deg,1.0,0.0,[stimulus_time_interval],"[(9, 1, timestamps pynwb.base.TimeSeries at 0x..."
8,186.854750,187.355136,init_grating,1.0,0.8,4.0,0.04,45.0,"[1.0, 1.0, 1.0]",None,1.0,"[1000.0, 1000.0]",deg,1.0,0.0,[stimulus_time_interval],"[(10, 1, timestamps pynwb.base.TimeSeries at 0..."


In [7]:
select_frame = 13.0

### LFP interpolation

In [ ]:
lfp = lfp_file.acquisition["probe_0_lfp_data"]

In [ ]:
stop_time = min(lfp.timestamps[-1],stim_timestamps[-1])
time_axis = np.arange(0,stop_time,step=0.001)
print(len(time_axis))

In [ ]:
lfp.timestamps.shape

In [ ]:
lfp.data.shape

In [ ]:
f = interpolate.interp1d(lfp.timestamps, lfp.data, axis=0, kind="nearest", fill_value="extrapolate")
intp_lfp = f(time_axis)
print((len(lfp.data)))
print(lfp.data[0:100])

print(len(intp_lfp))
print(intp_lfp[0:100])

In [ ]:
intp_lfp.shape

## LFP visualization

In [ ]:
node = 1
event_num = 100
start = -20
end = 300

In [ ]:
# get event windows

windows = []
for stim_ts in stim_timestamps:
    start_idx = int(stim_ts*1000) + start
    end_idx = int(stim_ts*1000) + end
    if start_idx < 0 or end_idx > len(intp_lfp)-1:
        continue
    windows.append(intp_lfp[start_idx:end_idx])

windows = np.array(windows)
print(windows.shape)

In [ ]:
average_trace = np.average(windows, axis=0)

In [ ]:
average_trace.shape

In [ ]:
1.96 * np.std(y)/np.sqrt(len(x))
ax.fill_between()

In [ ]:
# nodewise view
%matplotlib inline
fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(average_trace[:,])
# ax.plot(intp_lfp[10000:12000])
plt.show()